<a href="https://colab.research.google.com/github/gbardaklis/Mini-Project-2/blob/main/MP2_AA_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

order of importance, 
car, 
board, 
puzzle parser

In [54]:
# imports 
import copy
import time
import bisect

# Position Class 

In [55]:
class Position():
  def __init__(self, posX, posY):
    self.posX = posX
    self.posY = posY
  
  def __str__(self):
    return f"({str(self.posX)},{str(self.posY)})" 

# Car Class 

In [56]:
class Car(object):
  def __init__(self, name, position, fuel = 100, dir = "v"):
    self.name = name
    self.fuel = int(fuel)
    self.dir = dir
    self.positions = []
    self.positions.append(position)

  def getName(self) -> str:
    return self.name; 
    
  def isAmbulance(self) -> bool:
    return self.name == 'A'

  def isName(self, name) -> bool:
    return self.name == name

  # check if car has enough fuel to move, returns boolean
  def canMove(self) -> bool:
    return self.fuel > 0

  # may need to be changed
  def moveCarByLength(self, length):
    if (self.direction == (dir == "h")):
      for car in self.positions:
        car.posX += length
    else:
      for car in self.positions:
        car.posY += length
    self.fuel -= 1

  def updatePosition(self, updatePosition):
    self.positions.append(updatePosition)
    if self.positions[0].posX == updatePosition.posX and self.positions[0].posY != updatePosition.posY:
      self.direction = "h"
    else:
      self.direction = "v"

  def __str__(self) -> str:
    x = self.name + ' | '
    for iter in self.positions: 
      x += str(iter)
    return f" {x} | {self.fuel} | {self.direction}"

# Board Class 

In [57]:
class Board(object): 
  def __init__(self, cars):
    self.cars = []
    self.grid = [['.' for i in range(6)] for j in range(6)]
    self.size = {'X': 6, 'Y': 6}
    for car in cars:
      carHolder = car
      self.cars.append(copy.deepcopy(carHolder))
      # updating grid with the coordinate of the cars
      for position in carHolder.positions:
        # assign the letter at that position
        self.grid[position.posX][position.posY] = car.name

  def win(self):
    if self.grid[2][5] == 'A':
      return True
    else:
      return False

  def pathForward(self, car):
    direction = car.direction
    position = car.positions[len(car.positions)-1]
    if direction == 'h':
      head = position.posY
    else:
      head = position.posX
    
    moves = []
    for iter in range(head+1, 6):
      if car.fuel < iter-head:
        continue
      if direction == 'h':
        gridPosition = self.grid[position.posX][iter]
      else:
        gridPosition = self.grid[iter][position.posY]
      if gridPosition == '.':
        moves.append(iter-head)
      else:
        break
    return moves

  def pathBackward(self, car):
      direction = car.direction
      position = car.positions[0]
      if direction == 'h':
        head = position.posY
      else:
        head = position.posX
      
      moves = []
      for iter in range(head, 0, -1):
        if car.fuel <= head-iter:
          continue
        if direction == 'h':
          gridPosition = self.grid[position.posX][iter-1]
        else:
          gridPosition = self.grid[iter-1][position.posY]
        if gridPosition == '.':
          moves.append(iter-head-1)
        else:
          break
      return moves

  def findCarMoves(self, car):
    if car.canMove():
      forward = self.pathForward(car)
      backward = self.pathBackward(car)
      path = forward + backward
    else: 
      path = []
    return path

  def moveCar(self, name, length):
    for car in self.cars:
      if car.name == name:
        for position in car.positions:
          self.grid[position.posX][position.posY] = '.'
        car.moveCarByLength(length)
        for position in car.positions:
          self.grid[position.posX][position.posY] = name

        # car != ambulance reached exit, coord (2,5) h direction, game needs to continue
        if car.name == self.grid[2][5] and car.direction =='h' and car.name != 'A':
          for position in car.positions: 
            self.grid[position.posX][position.posY] = '.'
          self.cars.remove(car)

        direction = ""
        if length < 0 and car.direction == 'h':
          direction = 'left'
        if length > 0 and car.direction == 'h':
          direction = 'right'
        if length < 0 and car.direction == 'v':
          direction = 'down'
        if length > 0 and car.direction == 'v':
          direction = 'up'
        return name, direction, length, car.fuel, self.puzzle()

  # populate grid
  def puzzle(self):
    puzzle = ""
    for x in range(6):
      for y in range(6):
        puzzle += self.grid[x][y]
    return puzzle

  # grid default to string
  def __str__(self): 
    puzzle = ""
    for x in range(6):
      for y in range(6):
        puzzle += self.grid[x][y]
      puzzle += "\n" 
    return puzzle

  #heuristics
  #h1: number of blocking vehicles
  def h1(self):
    count = 0
    ambulance = self.grid[2]
    for x in range(6):
      if ambulance == 'A':
        continue
      elif ambulance != '.':
        count += 1
    return count
  
  #h2: number of blocking positions
  def h2(self):
    count = 0
    ambulance = self.grid[2]
    for x in range(6):
      if ambulance == 'A':
        continue
      elif ambulance != '.':
        count += 1
    return count

  #h3: value of h1 multiplied by a constant (lambda)
  def h3(self):
    constant = 3
    return self.h1() * constant

  #h4: your own admissible heuristic -- distance from ambulance to exit
  def h4(self):
    before = 0
    ambulance = self.grid[2]
    for x in range(6):
      if ambulance != 'A':
        before += 1
    return 4-before

# PuzzleParser Class 

In [58]:
class PuzzleParser:
  def createPuzzleList():
      input = open('/content/sample-input.txt', 'r')
      puzzles = []
      for line in input.readlines():
        if not line.startswith('#') and line.strip():
            puzzles.append(line.split(" ")) 
      input.close()
      return puzzles

# Testing Game Set Up

In [59]:
puzzler = PuzzleParser
puzzleMap = []
puzzles = puzzler.createPuzzleList();

for line in puzzles:
  grid = line[0];
  fuelLevel = []
  if len(line) > 1:
    fuelLevel.append(line[1:])

  cars = []
  for row in range(6):
    for col in range(6):
      current = grid[row*6 + col];
      if current == ".":
        continue
      isSame = True
      for car in cars:
        if car.isName(current):
          car.updatePosition(Position(row, col))
          isSame = False
      if isSame:
        fuelAmt = 100
        if len(fuelLevel) > 0:
          for fuel in fuelLevel[0]:
            if current == fuel[0]:
              fuelAmt = fuel[1:]

        cars.append(Car (name = current, position = Position(row,col), fuel = fuelAmt))

  board = Board(cars)
  print(board)
  for car in cars:
    print(car)
  print("---------------------------------")
  puzzleMap.append(board)

BBIJ..
..IJCC
..IAAM
GDDK.M
GH.KL.
GHFFL.

 B | (0,0)(0,1) | 100 | h
 I | (0,2)(1,2)(2,2) | 100 | v
 J | (0,3)(1,3) | 100 | v
 C | (1,4)(1,5) | 100 | h
 A | (2,3)(2,4) | 100 | h
 M | (2,5)(3,5) | 100 | v
 G | (3,0)(4,0)(5,0) | 100 | v
 D | (3,1)(3,2) | 100 | h
 K | (3,3)(4,3) | 100 | v
 H | (4,1)(5,1) | 100 | v
 L | (4,4)(5,4) | 100 | v
 F | (5,2)(5,3) | 100 | h
---------------------------------
..I...
BBI.K.
GHAAKL
GHDDKL
G..JEE
FF.J..

 I | (0,2)(1,2) | 100 | v
 B | (1,0)(1,1) | 100 | h
 K | (1,4)(2,4)(3,4) | 100 | v
 G | (2,0)(3,0)(4,0) | 100 | v
 H | (2,1)(3,1) | 100 | v
 A | (2,2)(2,3) | 100 | h
 L | (2,5)(3,5) | 100 | v
 D | (3,2)(3,3) | 100 | h
 J | (4,3)(5,3) | 100 | v
 E | (4,4)(4,5) | 100 | h
 F | (5,0)(5,1) | 100 | h
---------------------------------
JBBCCC
JDD..M
JAAL.M
FFKL.N
..KGGN
.HH...

 J | (0,0)(1,0)(2,0) | 100 | v
 B | (0,1)(0,2) | 100 | h
 C | (0,3)(0,4)(0,5) | 100 | h
 D | (1,1)(1,2) | 100 | h
 M | (1,5)(2,5) | 100 | v
 A | (2,1)(2,2) | 100 | h
 L | (2,3)(3,3) | 1

# Node Classes

In [60]:
class Node(object):
  # initialize
  def __init__(self, pathcost, board):
    self.parent = 0
    self.pathcost = pathcost
    # cost of the edge
    self.heuristiccost = 1
    # path + h cost
    self.totalcost = 0 
    self.board = board
    self.path = []
    self.search = ""

  #less than
  def __lessthan__(self, other):
    return self.pcost < other.pcost

  #equal to
  def __equalto__(self, other):
    return self.board == other.board

# heuristic node
class HNode(object):
  def __init__(self, board, cost, parentCost):
    self.board = board
    # g(n)
    self.cost = cost 
    # the parent node, if root then -1 because it increments 
    self.parentCost = parentCost
    # h(n)
    self.heuristiccost = 0 #initialized to 0
    # Assign the Parent Node to the child Node Object when making child Nodes
    self.parentNode = None
    # Holds a path of movements 
    self.path = []



  def __lessthan__(self, other) -> bool:
    return self.cost < other.cost

  def __equalto__(self, other) -> bool:
    return self.board == other.board

  # same as dan 
  def setHeuristic(self, h):
    if h == 1:
        self.cost = self.cost + self.board.h1() 
    if h == 2:
         self.cost = self.cost + self.board.h2() 
    if h == 3:
         self.cost = self.cost + self.board.h3() 
    if h == 4:
         self.cost = self.cost + self.board.h4() 
    return self.cost

  def setCost(self, cost):
    self.cost = cost

#Uniform Cost Search (UCS)

In [ ]:
#note: h(n) irrelevant for ucs therefore will always be 0
#note: g(n) irrelevant for gbfs therefore will always be 0
#note: g(n) irrelevant for a/a* therefore will always be 0

class UCS:
  def __init_(self):
    self.open = []
    self.close = []
    self.puzzlenumber = 0

  # for ucs-search-{x}.txt
  def search(self, board):
    start = time.time() 
    self.open = []
    self.close = []
    puzzlenumber += 1

    #node is created with initial state, null parent, null action and path cost of 0
    startnode = Node(copy.deepcopy(board), 0)
    self.open.append(startnode)
    #set initial f(n), g(n) and h(n) to 0 for the startnode
    startnode.search = "0 0 0 " + startnode.board.puzzle()
    
    #write to search files f(n), g(n), h(n) values of each node
    searchfiles = open('/content/output/' + 'ucs-search-' + str(self.puzzlenumber) + '.txt', 'w')

    #set state counter

    #write to solution files
    while(True):
      if(len(self.open) == 0):
        print("empty")
        searchfiles.close() #close because nothing to write to search file if empty

      solutionfiles = open('/content/output/' + 'ucs-sol-' + str(self.puzzlenumber) + '.txt', 'w')
      solutionfiles.write("Initial board configuration: " + board.puzzle() + "\n\n")
      solutionfiles.write(board + "\n")

# AA* TEST

In [61]:
class AAStar(object):
  def __init__(self):
    self.openList = []
    self.closedList = []
    # state counter
    self.counter = 0 

  def incrementState(self):
    self.counter += 1
  
  def printNoSol(self, board, h):
    #writing for no sol  
    solaaStar = open('/content/' + 'a-h' + str(h) + '-sol-' + str(self.counter) + '.txt', 'w')
    solaaStar.write(f"Initial board configuration: {str(board.puzzle())} \n\n")
    solaaStar.write(f"{str(board)} \n")
    solaaStar.write(f"Car fuel available: ")
    for iter in range(len(board.cars)): 
      # no one cares about removing a coma
      solaaStar.write(f"{str(board.cars[iter].name)}:{str(board.cars[iter].fuel)}, ")
    solaaStar.write("No Solution")
    solaaStar.close()
    return

  def printSol(self, board, h, startTime, states, currentNode):
    solaaStar = open('/content/' + 'a-h' + str(h) + '-sol-' + str(self.counter) + '.txt', 'w')
    solaaStar.write(f"Initial board configuration: {str(board.puzzle())} \n\n")
    solaaStar.write(f"{str(board)} \n")
    solaaStar.write(f"Car fuel available: ")
    for iter in range(len(board.cars)): 
      # no one cares about removing a coma
      solaaStar.write(f"{str(board.cars[iter].name)}:{str(board.cars[iter].fuel)}, ")
    runtime = time.time() - startTime
    solaaStar.write(f"Runtime: {str(runtime)} seconds \n")
    solaaStar.write(f"Search path length: {str(states)} states \n") #large number
    # solaaStar.write(f"Solution path length: ") # smaller num 
    solaaStar.write(f"Solution Path: \n")
    for pathIndex in currentNode.path:
      solaaStar.write(f"{pathIndex[0]} \t {pathIndex[1]} \t {pathIndex[2]} \t {pathIndex[3]} \t {pathIndex[4]} \n")
    solaaStar.write(f"{str(currentNode.board)} \n")
    solaaStar.close()
    return

  def compareCost(self, childNode, openList, closedList):
    if childNode in self.openList:
      for nodeCheck in self.openList:
        if nodeCheck == childNode and nodeCheck.cost > childNode.cost:
          self.openList.remove(nodeCheck)
          # create a placeholder to insert using bisect for a list 
          bisect.insort(self.openList, childNode)
          break
        continue
    elif childNode in self.closedList:
      for nodeCheck in self.closedList:
        if nodeCheck == childNode and nodeCheck.cost > childNode.cost:
          self.closedList.remove(nodeCheck)
          bisect.insort(self.openList, childNode)
          break
        continue
    else:
      bisect.insort(self.openList, childNode) 

  def aastarSearch(self, board, h): # h is heuristic
    startTime = time.time() # runtime = time.time() (at run time) - startTime
    states = 0 # set up a counter for seach path lengh 

    # board = copy.deepcopy(board)
    
    startNode = HNode(copy.deepcopy(board), 0, -1) # cost =0, parent node cost =-1 because theres no cost, children then it increments and gets set to 0  
    self.openList.append(startNode) # append startnode to open list 

    # h tbd when you call the class and function 1234
    startNode.setHeuristic(h)
    
    # f(n), g(n) and h(n) respectively, f=g+h, g start is default 0
    startNode.aastarSearch = str(startNode.cost) + " 0 " + str(startNode.cost) + " " + startNode.board.puzzle()

    searchaastar = open('/content/' + 'a-h' + str(h) + '-search-' + str(self.counter) + '.txt', 'w')

    # needs to be in some kind of loop 
    while(True):
      if (len(self.openList)==0):
        print("inside no sol")
        # not win print func
        self.printNoSol(board, h)

      # pop off node
      currentNode = self.openList.pop(0)
      currentNode.setCost(currentNode.parentCost + 1)
      # update the search on file
      searchaastar.write(f"{currentNode.aastarSearch} \n")

      if currentNode.board.win():
        print("inside a possible sol")
        self.printSol(board, h, startTime, states, currentNode)

      # CurrentNode is not satisfactory move to closedList
      self.closedList.append(currentNode)
      states += 1

      for car in currentNode.board.cars:
        pathsChecked = currentNode.board.findCarMoves(car)
        if not pathsChecked:
          for iter in range (len(pathsChecked)):
            
            #Creating a successor node 
            childNode = HNode(Board(currentNode.board.cars), currentNode+1, currentNode.cost)
            childNode.parentNode = currentNode
            #Create a deep Copy of Child Node Path from parent Node
            childNode.path = copy.deepcopy(childNode.parentNode.path)

            displacement = childNode.board.moveCar(car.name, pathsChecked[iter])
            childNode.path.append(displacement)
            combinedCost = childNode.cost
            childNode.setHeuristic(h)
            childNode.aastarSearch = str(childNode.cost) + " 0 " + str(childNode.cost) + " " + childNode.board.puzzle()

            self.compareCost(childNode, self.openList, self.closedList)
      
  



In [63]:
aaStar = AAStar()

for puzzles in puzzleMap: 
  currentBoard = Board(puzzles.cars)
  print(currentBoard)
  aaStar.incrementState()
  for i in range(1,5):
    aaStar.aastarSearch(currentBoard, 1)

BBIJ..
..IJCC
..IAAM
GDDK.M
GH.KL.
GHFFL.

inside no sol


IndexError: ignored

# LIST OF UNUSED FUCTIONS ❗ 💯

This function returns a list of cars with their FUEL from the list of puzzles

In [ ]:
# this should become a PuzzleParser Class 
def __create_puzzle_list():
    input = open('/content/sample-input-2.txt', 'r')
    puzzles = []
    for line in input.readlines():
      if not line.startswith('#') and line.strip():
          puzzles.append(line.strip()) 
    input.close()
    return puzzles
  
  # seperates the fuel from the board 
def __get_fuel():
    # a list of cars temp
    carsTemp = []
    # a list of car objects
    cars = []
    for x in puzzles:
      puzzle_str = x.split(" ")
      # if only 0 then theres only a puzzle, default 100
      # else fuel assigned
      if len(puzzle_str) > 0:
        for i in range (1, len(puzzle_str)):
          carsTemp.append(puzzle_str[i])
          carSample = carsTemp[i-1]
          carSampleLetter = carSample[0]
          carSampleFuel = carSample[1]
          cars.append(Car (name = carSampleLetter, fuel = carSampleFuel))
        return cars

In [ ]:
puzzles = __create_puzzle_list();

In [ ]:
a = __get_fuel()
print(a[0].name)

Writes to a file the desired layout with the fuel and car name

In [ ]:
# def __output():
output = open("/content/test.txt", "w")
for x in puzzles:
  output.write(f"Initial board configuration: {x} \n\n")
  splitted_line = x.split(" ")
  puzzlestr = splitted_line[0]
  count = 0
  for i in range(len(puzzlestr)):
    output.write(puzzlestr[i] + " ")
    count += 1
    if count == 6:
      output.write("\n")
      count = 0
  output.write("\n")
  output.write("Car fuel available: ")
  for f in a: 
    output.write(f"{f.name}:{f.fuel}, ")

This takes in a puzzle string and counts the occurrences of the char to make the SIZE of the car

In [ ]:
def __checkSize(puzzle): 
  cars = []
  # Test case
  # input_string = "BBIJ....IJCC..IAAMGDDK.MGH.KL.GHFFL."
  mySet = set(puzzle)
  countOfChars = dict()
  for element in mySet:
      countOfChar = puzzle.count(element)
      countOfChars[element] = countOfChar
      if (element == "."):
        continue;
      else:  
        cars.append(Car (name = element, size = countOfChar))

  counter = 0;
  for x in cars:
    print (cars[counter].name + str(cars[counter].size) ) 
    counter += 1;  
  return cars

Checks if a car is setup HORIZONTALLY

In [ ]:
def __checkHorizontal(puzzle): 
  uniqueHorizontal = []
  for curr,next in zip(puzzle, puzzle[1:]): 
    if (curr == "."):
      continue;
    if (curr == next) :
      uniqueHorizontal.append(curr)
    else:
      continue;
  finalHorizontal = set(uniqueHorizontal)    
  return finalHorizontal

This is the Check for Horizontal and Size Functions

In [ ]:
newString = puzzles[0]
newString = newString[:36]
returnedHorizontalCar = __checkHorizontal(newString)
print(returnedHorizontalCar)

# print('\n' + 'this is the checksize function')
returnedCarSize = __checkSize(newString)
print(returnedCarSize)

TO REVISIT 

In [ ]:
#Combining Both lists
counter2=0
for (currentCar, horizontalCar) in zip(returnedCarSize, returnedHorizontalCar):

  thisIsTheCar = returnedCarSize[counter2].name
  if ( thisIsTheCar != horizontalCar):
    print("this is not a match" + thisIsTheCar + horizontalCar)
  if ( thisIsTheCar == horizontalCar):
    returnedCarSize[counter2].dir = "h"
    counter2+=1  
  else:
    counter2+=1


counter = 0;
for x in returnedCarSize:
    print (returnedCarSize[counter].name + str(returnedCarSize[counter].size) + returnedCarSize[counter].dir) 
    counter += 1;  
# print(returnedCarSize[0].dir) 


Slices the puzzle string into 36 chars (6x6) passes into ndarray 

In [ ]:
newString = puzzles[0]
newString = newString[:36]
print(newString)

test = np.array(list(newString)).reshape(6,6)
print(test)
print(test[0][4])
print(type(test))



BBB..MCCDD.MAAKL.MJ.KLEEJ.GG..JHHHII
[['B' 'B' 'B' '.' '.' 'M']
 ['C' 'C' 'D' 'D' '.' 'M']
 ['A' 'A' 'K' 'L' '.' 'M']
 ['J' '.' 'K' 'L' 'E' 'E']
 ['J' '.' 'G' 'G' '.' '.']
 ['J' 'H' 'H' 'H' 'I' 'I']]
.
<class 'numpy.ndarray'>


In [ ]:
board1 = Board(board = "BBB..MCCDD.MAAKL.MJ.KLEEJ.GG..JHHHII")
print(board1.board)

BBB..MCCDD.MAAKL.MJ.KLEEJ.GG..JHHHII


In [ ]:
board2 = Board(board = puzzles[0])
print(board2.board)

BBIJ....IJCC..IAAMGDDK.MGH.KL.GHFFL.


In [ ]:
# def __play_rush_hour_ucs():
#   # returns list of puzzles 
#   puzzles = __create_puzzle_list();
#   for index, input in enumerate(puzzles):
#     __puzzle_output_ucs(input, f"ucs-sol-{index+1}.txt")


In [ ]:
# NOT GOING TO BE USED 
# Makes a DICT of occurrences and chars
test = "BBIJ....IJCC..IAAMGDDK.MGH.KL.GHFFL."

letterDict = dict()

# print(letterDict)

letterList = []

for x in test:
  occurences = test.count(x)
  # print(occurences)
  if (x == ".") or (x in letterList):
    continue; 
  else:
    letterList.append(f"{x} + {occurences}")
    
finalList = set(letterList)
print(finalList)

{'G + 3', 'A + 2', 'D + 2', 'I + 3', 'C + 2', 'H + 2', 'F + 2', 'B + 2', 'J + 2', 'K + 2', 'M + 2', 'L + 2'}
